In [1]:
import torch

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    device = "cuda"
    print("CUDA device:", torch.cuda.get_device_name(0))
    print("Device count:", torch.cuda.device_count())
else:
    device = "cpu"
    print("Running on CPU only.")

Torch version: 2.7.1+cu118
CUDA available: True
CUDA device: NVIDIA GeForce RTX 3060 Laptop GPU
Device count: 1


In [2]:
# init
model_path = './orchestrator-model/TinyLlama-1.1B-Chat-v1.0/'

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
model.to(device)

tokenizer = AutoTokenizer.from_pretrained(model_path)

input_text = "What's your task?"
inputs = tokenizer(input_text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


What's your task?

JASON: (smiling) I'm here to help you with your homework.

SARAH: (smiling back) That's right. I'm doing a report on the history of the city.

JASON: (nodding) That's a great start. Can you tell me more about the city's history?

SARAH: (thinking) Well, the city was founded in


## Preprocess Raw Data

In [4]:
import preprocess_dataset as pre
from datasets import Dataset

#init
dataset_root = "./datasets/raw/en/"
dataset_save_root = "./datasets/formatted/en/"

### Tool Memorization and Classification Dataset Preprocess

#### 1.  send_email tool dataset preprocessing

In [5]:
send_email_dataset_raw : Dataset = pre.load_jsonl_dataset(dataset_root + "send_email.jsonl")
send_email_dataset_formatted : dict = pre.format_dataset(send_email_dataset_raw)

for i in send_email_dataset_formatted:
    print(i)

{'instruction': 'Send an email to my manager saying the report is done.', 'response': '{"action": "send_email", "args": {"recipient": "<recipient_email>", "subject": "<email_subject>", "body": "<email_body>"}}', 'prompt': '### Instruction:\nSend an email to my manager saying the report is done.\n\n### Response:'}
{'instruction': 'Email the supplier to confirm the shipment.', 'response': '{"action": "send_email", "args": {"recipient": "<supplier_email>", "subject": "Shipment Confirmation", "body": "Hello, could you please confirm the status of the shipment for order <order_id>?"}}', 'prompt': '### Instruction:\nEmail the supplier to confirm the shipment.\n\n### Response:'}
{'instruction': 'Send my resignation letter to HR.', 'response': '{"action": "send_email", "args": {"recipient": "<hr_email>", "subject": "Resignation Letter", "body": "Dear HR, I am submitting my resignation effective from <resignation_date>. Please let me know the next steps."}}', 'prompt': '### Instruction:\nSend m

In [6]:
# save
send_email_formatted_output_path = dataset_save_root + "send_email.jsonl"

saving_send_email_dataset_isSuccess : bool = pre.save_dataset(send_email_dataset_formatted, send_email_formatted_output_path)
if saving_send_email_dataset_isSuccess :
    print("Successfully saved dataset for sending email tool classification")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Successfully saved dataset for sending email tool classification


## Train Model

In [7]:
#init
from transformers import AutoTokenizer

model_path = "./orchestrator-model/TinyLlama-1.1B-Chat-v1.0/" # or HuggingFace model ID
tokenizer = AutoTokenizer.from_pretrained(model_path)


### Tool Memorization and Classification Model Training

#### 1. send_email tool classification training

In [8]:
# Tokenization
def tokenize_function(example):
    result = tokenizer(
        example["prompt"] + " " + example["response"],
        truncation=True,
        max_length=512,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

tokenized_dataset = send_email_dataset_formatted.map(tokenize_function, remove_columns=send_email_dataset_formatted.column_names)


In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./orchestrator-model-finetuned",
    per_device_train_batch_size=1,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    remove_unused_columns=True,
    fp16=True,
    gradient_accumulation_steps=4
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=False
)


In [10]:
print(tokenized_dataset[0].keys())

dict_keys(['input_ids', 'attention_mask', 'labels'])


In [11]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


C:\Users\Fadhl\AppData\Local\Temp\ipykernel_9684\1889173795.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


KeyboardInterrupt: 